In [23]:
import psycopg2
import pandas as pd
import geopandas as gpd
import json
import folium
import branca.colormap as cm
from config import db_config

In [24]:
# Load the GeoJSON file with county boundaries
geojson_file_path = 'Boundaries.geojson'  

# Load the GeoJSON data
with open(geojson_file_path) as f:
    geojson_data = json.load(f)

In [25]:
# Function to connect to PostgreSQL and retrieve data
def get_data_from_postgres(query):
    try:
        # Use the credentials from config.py
        connection = psycopg2.connect(
            host=db_config['host'],
            database=db_config['database'],
            user=db_config['user'],
            password=db_config['password'],
            port=db_config['port']
        )
        cursor = connection.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(data, columns=columns)
        return df

    except Exception as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [26]:
# Query to fetch data for the years 2020, 2021, 2022, and 2023
query = """
    SELECT countyname, year, deaths, geometry 
    FROM overdose_deaths 
    WHERE year IN (2020, 2021, 2022, 2023)
"""
overdose_data = get_data_from_postgres(query)

# Iterate over the rows of the DataFrame
for index, row in overdose_data.iterrows():
    # Extract the relevant information
    countyname = row['countyname']
    deaths = row['deaths']
    year = row['year']
    geometry = row['geometry']  # Get the year from the dataset

    # Print the result with the county, year, deaths, and coordinates
    print(f"County: {countyname}, Year: {year}, Deaths: {deaths}, Geometry: {geometry}")


County: Adams, Year: 2020, Deaths: 132, Geometry: POLYGON ((-83.271 39.016, -83.268 38.861, -83.268 38.616, -83.28700000000001 38.6, -83.307 38.599, -83.319 38.609, -83.33 38.639, -83.37 38.659, -83.398 38.664, -83.45999999999999 38.671, -83.503 38.698, -83.523 38.702, -83.539 38.701, -83.58199999999999 38.69, -83.61199999999999 38.685, -83.628 38.676, -83.636 38.661, -83.642 38.636, -83.657 38.626, -83.66800000000001 38.626, -83.705 38.639, -83.697 38.762, -83.673 39.02, -83.61199999999999 39.018, -83.512 39.037, -83.386 39.055, -83.315 39.052, -83.306 39.043, -83.271 39.016))
County: Adams, Year: 2021, Deaths: 188, Geometry: POLYGON ((-83.271 39.016, -83.268 38.861, -83.268 38.616, -83.28700000000001 38.6, -83.307 38.599, -83.319 38.609, -83.33 38.639, -83.37 38.659, -83.398 38.664, -83.45999999999999 38.671, -83.503 38.698, -83.523 38.702, -83.539 38.701, -83.58199999999999 38.69, -83.61199999999999 38.685, -83.628 38.676, -83.636 38.661, -83.642 38.636, -83.657 38.626, -83.66800000

In [27]:
# Split the data by year
data_2020 = overdose_data.loc[overdose_data["year"] == 2020]
data_2021 = overdose_data.loc[overdose_data["year"] == 2021]
data_2022 = overdose_data.loc[overdose_data["year"] == 2022]
data_2023 = overdose_data.loc[overdose_data["year"] == 2023]


In [28]:
# Initialize map over Ohio
map = folium.Map(location=[39.9612, -82.9988], zoom_start=8)

# Creates colormap scale and legend
colormap = cm.linear.YlOrRd_05.scale(0, 10000).to_step(10)
colormap.caption = "Provisional Drug Overdose Deaths"
map.add_child(colormap)

# Function to create each overlay layer
def yearOverlay(name, data, startingOverlay):
    # Convert the deaths column to Python integers
    data['deaths'] = data['deaths'].astype(int)
    
    # Creates series of death counts with county name as index. Used for colormap.
    series = data.set_index("countyname")["deaths"]

    # Update GeoJSON data with death counts
    for feature in geojson_data['features']:
        county_name = feature['properties']['name']
        feature['properties'][name] = int(series.get(county_name, 0))  # Ensure the value is a Python int
    
    # Creates overlay layer using updated GeoJSON data
    folium.GeoJson(geojson_data,
               name=name,
               show=startingOverlay,
               style_function=lambda feature: {
                   "fillColor": colormap(feature["properties"].get(name, 0)),  # Use `.get` to handle missing counties
                   "fillOpacity": .6,
                   "color": "black",
                   "weight": 2
               },
               popup=folium.GeoJsonPopup(fields=["name", name], aliases=["County", "Deaths"]),
               popup_keep_highlighted=True,
               tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["County"])
               ).add_to(map)

# Run function for each year of data
yearOverlay("2020", data_2020, True)
yearOverlay("2021", data_2021, False)
yearOverlay("2022", data_2022, False)
yearOverlay("2023", data_2023, False)

# Add layer control
folium.LayerControl().add_to(map)

# Save the map to an HTML file
map.save('ohio_overdose_map.html')

map

C:\Users\Scarthicus\AppData\Local\Temp\ipykernel_20788\4165375061.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['deaths'] = data['deaths'].astype(int)
C:\Users\Scarthicus\AppData\Local\Temp\ipykernel_20788\4165375061.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['deaths'] = data['deaths'].astype(int)
C:\Users\Scarthicus\AppData\Local\Temp\ipykernel_20788\4165375061.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in